##  [&#8593;](#toc0_) <a id='toc1_1_'></a>**Inteligência Geográfica**
____________________________________________________________

Tabela de Entrada:

            idescola    ano    idserie    endereco

**Table of contents**<a id='toc0_'></a>    
- [**Inteligência Geográfica**](#toc1_1_)    
    - [**Configurações**](#toc1_1_1_)    
      - [Import](#toc1_1_1_1_)    
      - [Conta de Pagamento](#toc1_1_1_2_)    
      - [Variáveis](#toc1_1_1_3_)    
    - [**Extract**](#toc1_1_2_)    
      - [Dicionários](#toc1_1_2_1_)    
      - [Base](#toc1_1_2_2_)    
    - [**Geocode**](#toc1_1_3_)    
    - [**Saída**](#toc1_1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

###  [&#8593;](#toc0_) <a id='toc1_1_1_'></a>**Configurações**

####  [&#8593;](#toc0_) <a id='toc1_1_1_1_'></a>Import

In [1]:
import pandas as pd
import sys
import warnings
import os

from datetime import datetime
from geocoder import Geocoder 

sys.path.append('/apoiodev/')
from connect import conn, command_bcp, update_pbi, gmaps, ws_inteligencia, ds_inteligencia
from config import cd_Bcp, cd_GeoData, cd_CacheGeo
from utils import clean_text
warnings.filterwarnings("ignore")

####  [&#8593;](#toc0_) <a id='toc1_1_1_3_'></a>Variáveis

In [2]:
idescola = input("idescola (0 para todas): ")
ano_inicial = input("digite o ano inicial: ")
ano_final = input("digite o ano final: ")


###  [&#8593;](#toc0_) <a id='toc1_1_2_'></a>**Extract**

####  [&#8593;](#toc0_) <a id='toc1_1_2_1_'></a>Dicionários

In [3]:
def get_sqlGeoAuditoria(idescola):
    auditoria = pd.read_sql(f"SELECT * FROM [audita_geo_alunos] ('{idescola}') ORDER BY ano;", conn) 
    return auditoria


def get_sql_dict(column_name):    
    query = f'SELECT idpoint, {column_name} FROM [cadastro_d_points];'
    dPoints = pd.read_sql(query, conn)
    sql_dict = dPoints.set_index(column_name)['idpoint'].to_dict()
    return sql_dict
    

####  [&#8593;](#toc0_) <a id='toc1_1_2_2_'></a>Base

In [4]:
if idescola == '0':
    dfs = []
    for arquivo in os.listdir(cd_GeoData):
        if arquivo.endswith(".xlsx"):
            df = pd.read_excel(os.path.join(cd_GeoData, arquivo))
            dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
else:
    df = pd.DataFrame(pd.read_excel(f"{cd_GeoData}{idescola}.xlsx"))

df['ano'] = df['ano'].astype(int)
df = df.loc[
    (df['ano'] >= int(ano_inicial)) & 
    (df['ano'] <= int(ano_final)) & 
    (df['idserie'] != 27)].reset_index(drop=True)
print(f"df: {len(df)} linhas")
df['endereco'] = df['endereco'].apply(lambda x: clean_text(x))

df: 175 linhas


###  [&#8593;](#toc0_) <a id='toc1_1_3_'></a>**Geocode**

#### Busca endereços

- **Primeira Busca:**  procura por enderecos já mapeados em [cadastro_d_points] e retorna o idpoint

In [5]:
# Realiza a primeira busca dos idpoints na base SQL, a partir do endereco.
enderecos_dict = get_sql_dict('endereco')
point_dict = get_sql_dict('point')

df['idpoint'] = df['endereco'].map(enderecos_dict)

- **Segunda Busca:** procura os enderecos não cadastrados na base no diretório de cache do trabalho (cd_CacheGeo)

In [6]:
if len(df.loc[df['idpoint'].isna()]) > 0:
    caches = []
    for arquivo in os.listdir(cd_CacheGeo):
        if arquivo.endswith(".csv"):
            cache = pd.read_csv(os.path.join(cd_CacheGeo, arquivo))
            caches.append(cache)
    cache = pd.concat(caches, ignore_index=True)
    cache = cache.drop_duplicates().reset_index(drop=True)
    cache['point'] = (cache['latitude'].astype(str) + ', ' + cache['longitude'].astype(str)).str.replace('.',',')
    point_dict = get_sql_dict('point')
    cache['idpoint'] = cache['point'].map(point_dict)
    cache_dict = cache.set_index('endereco')['idpoint'].to_dict()

    df.loc[df['idpoint'].isna(), 'idpoint'] = df['endereco'].map(cache_dict)

#### Cadastra endereços

- **Geoprocessamento:**  geoprocessa os enderecos ainda não cadastrados na base

In [7]:
base_geocode = pd.DataFrame(df.loc[df['idpoint'].isna()]['endereco'].drop_duplicates().reset_index(drop=True).copy())

print(f"{len(base_geocode)} endereços encaminhados para geoprocessamento")
if not base_geocode.empty:  
    print(f"geoprocessamento iniciado em {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")



Geocode = Geocoder(base_geocode)
end_geocoded = Geocode.Processar()



if not base_geocode.empty:  
    print(f"geoprocessamento finalizado em {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{len(end_geocoded)} endereços geoprocessados")

155 endereços encaminhados para geoprocessamento
geoprocessamento iniciado em 2024-03-14 11:43:15
geoprocessamento finalizado em 2024-03-14 11:43:21
155 endereços geoprocessados


- confere se os enderecos geoprocessados já constam na base a partir do point

In [8]:
if not base_geocode.empty:  
    
    geocoded = end_geocoded.loc[end_geocoded['latitude'].notna()].reset_index().copy()
    print(f"{len(geocoded)} endereços válidos geoprocessados")
    geocoded['point'] = (geocoded['latitude'].astype(str) + ', ' + geocoded['longitude'].astype(str)).str.replace('.',',')
    geocoded['idpoint'] = geocoded['point'].map(point_dict)

155 endereços válidos geoprocessados


- cadastra os novos endereços

In [9]:
if not base_geocode.empty:  

    Cadastrar = geocoded.loc[geocoded['idpoint'].isna()].reset_index(drop=True).copy()
    print(f'points não cadastrados em cadastro_d_points:{len(Cadastrar)}') 

    if len(Cadastrar) > 0:

        new_idpoint = max(point_dict.values()) + 1

        for idx, row in Cadastrar.iterrows():
            Cadastrar.at[idx, 'idpoint'] = new_idpoint          
            point_dict[row['point']] = new_idpoint
            new_idpoint += 1

        points_to_update = Cadastrar.set_index('endereco')['idpoint'].to_dict()
        df.loc[df['idpoint'].isna(), 'idpoint'] = df['endereco'].map(points_to_update)

        Cadastrar = Cadastrar.reindex(columns=['idpoint', 'point', 'bairro', 'latitude', 'longitude', 'address', 'cidade', 'uf', 'cep'])

        Cadastrar['idpoint'] = Cadastrar['idpoint'].astype(int)
        Cadastrar['latitude'] = Cadastrar['latitude'].astype(float)
        Cadastrar['longitude'] = Cadastrar['longitude'].astype(float)

        bcp_file = cd_Bcp + 'd_points.csv'
        if os.path.isfile(bcp_file):
            os.remove(bcp_file)

        Cadastrar.to_csv(bcp_file, sep=';', encoding='utf-8', index=False)
        result = command_bcp('dbo', 'cadastro_d_points', bcp_file)           
    else:
        result = "Comando bcp nao executado"

    print(result)

    geocoded2 = geocoded.loc[geocoded['idpoint'].notna()].reset_index(drop=True).copy()
    base_geocodificada = pd.concat([geocoded2, Cadastrar], ignore_index=True)

    print(f"Resultado: {len(base_geocodificada)} enderecos mapeados a serem inseridos no df")

points não cadastrados em cadastro_d_points:134

Iniciando cópia...

134 linhas copiadas.
Tamanho do pacote de rede (bytes): 4096
Tempo total do relógio (ms.)     : 63     Média : (2126.98 linhas /s.)

Resultado: 155 enderecos mapeados a serem inseridos no df


- **Alimenta o df principal com os dados geocodificados**

In [10]:
if not base_geocode.empty:  

    base_geocodificada_dict = base_geocodificada.set_index('endereco')['idpoint'].to_dict()
    for endereco, idpoint in base_geocodificada_dict.items():
        df.loc[df['endereco'] == endereco, 'idpoint'] = idpoint

print(f"df: {len(df)} linhas")

df: 175 linhas


###  [&#8593;](#toc0_) <a id='toc1_1_4_'></a>**Saída**

In [11]:
df_saida = df.loc[df['idpoint'].notna()].reset_index(drop=True).copy()
df_saida['iddata'] = pd.to_datetime(df_saida['ano'].astype(str) + '-03-01')
df_saida['idpoint'] = df_saida['idpoint'].astype(str).str.replace('.0', '')
df_saida['idpoint'] = df_saida['idpoint'].astype(int)
df_saida = df_saida[['idescola', 'iddata', 'idserie', 'idpoint']]

In [12]:
list_idescola = list(df_saida['idescola'].unique())

for id in list_idescola:
    saida = df_saida.loc[df_saida['idescola'] == id].reset_index(drop=True).copy()
    bcp_file = f"{cd_Bcp}{id}{ano_final}_geo.csv"
    saida.to_csv(bcp_file, sep=';', encoding='utf-8', index=False)
    result = command_bcp('dbo', 'geo_f_alunos', bcp_file)
    MsgPBI = update_pbi(ws_inteligencia, ds_inteligencia)
    print(f"Resultado PBI: {MsgPBI}")
    print(f'Escola:{id}')
    print("______________________")
    print(result)
    print("Auditoria:")
    auditoria = get_sqlGeoAuditoria(id) 
    
    display(auditoria)

Resultado PBI: Solicitação de atualização enviada com sucesso.
Escola:C9271
______________________

Iniciando cópia...

175 linhas copiadas.
Tamanho do pacote de rede (bytes): 4096
Tempo total do relógio (ms.)     : 203    Média : (862.07 linhas /s.)

Auditoria:


,ano,AlunosGeo,AlunosDE
0,2022,181,181
1,2024,175,175
